In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import pygame
import random
import sys

In [40]:
# Initialize Pygame
pygame.init()

# Load the image
image = pygame.image.load('lima.jpg')
image_rect = image.get_rect()

# Define puzzle dimensions
rows, cols = 3, 3  # Adjust the number of rows and columns as needed
piece_width = image_rect.width // cols
piece_height = image_rect.height // rows

# Create the game window
screen = pygame.display.set_mode((image_rect.width, image_rect.height))
pygame.display.set_caption('Jigsaw Puzzle')

In [41]:
# Function to create a mask for a puzzle piece
def create_piece_mask(row, col):
    mask = pygame.Surface((piece_width, piece_height), pygame.SRCALPHA)
    mask.fill((255, 255, 255, 0))  # Transparent background

    # Define tab/slot size
    tab_size = 20  # Adjust as needed

    # Draw the piece with tabs and slots
    points = [(0, 0)]

    # Top edge
    if row == 0:
        points.append((piece_width, 0))
    else:
        points.extend([
            (piece_width // 3, 0),
            (piece_width // 3 + tab_size, -tab_size),
            (2 * piece_width // 3 - tab_size, -tab_size),
            (2 * piece_width // 3, 0),
            (piece_width, 0)
        ])

    # Right edge
    if col == cols - 1:
        points.append((piece_width, piece_height))
    else:
        points.extend([
            (piece_width, piece_height // 3),
            (piece_width + tab_size, piece_height // 3 + tab_size),
            (piece_width + tab_size, 2 * piece_height // 3 - tab_size),
            (piece_width, 2 * piece_height // 3),
            (piece_width, piece_height)
        ])

    # Bottom edge
    if row == rows - 1:
        points.append((0, piece_height))
    else:
        points.extend([
            (2 * piece_width // 3, piece_height),
            (2 * piece_width // 3 - tab_size, piece_height + tab_size),
            (piece_width // 3 + tab_size, piece_height + tab_size),
            (piece_width // 3, piece_height),
            (0, piece_height)
        ])

    # Left edge
    if col == 0:
        points.append((0, 0))
    else:
        points.extend([
            (0, 2 * piece_height // 3),
            (-tab_size, 2 * piece_height // 3 - tab_size),
            (-tab_size, piece_height // 3 + tab_size),
            (0, piece_height // 3),
            (0, 0)
        ])

    pygame.draw.polygon(mask, (255, 255, 255, 255), points)
    return mask

In [42]:
# # Create puzzle pieces
# pieces = []
# for row in range(rows):
#     for col in range(cols):
#         piece_image = pygame.Surface((piece_width, piece_height), pygame.SRCALPHA)
#         piece_image.blit(image, (0, 0), (col * piece_width, row * piece_height, piece_width, piece_height))
#         pieces.append({
#             'image': piece_image,
#             'position': (col * piece_width, row * piece_height),
#             'correct_position': (col * piece_width, row * piece_height)
#         })
#         print(f"Piece {len(pieces)}: position {pieces[-1]['position']}")
# Create puzzle pieces
pieces = []
for row in range(rows):
    for col in range(cols):
        piece_image = pygame.Surface((piece_width, piece_height), pygame.SRCALPHA)
        piece_image.blit(image, (0, 0), (col * piece_width, row * piece_height, piece_width, piece_height))
        mask = create_piece_mask(row, col)
        piece_image.blit(mask, (0, 0), None, pygame.BLEND_RGBA_MULT)
        pieces.append({
            'image': piece_image,
            'position': (random.randint(0, image_rect.width - piece_width), random.randint(0, image_rect.height - piece_height)),
            'correct_position': (col * piece_width, row * piece_height),
            'row': row,
            'col': col
        })

In [38]:
# Shuffle the pieces
random.shuffle(pieces)

# Assign new positions
for i, piece in enumerate(pieces):
    piece['position'] = (random.randint(0, image_rect.width - piece_width), random.randint(0, image_rect.height - piece_height))


In [43]:
# Variables to track dragging state
dragging = False
dragged_piece = None
offset_x = 0
offset_y = 0
snap_threshold = 20  # Pixels

# Main game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            mouse_x, mouse_y = event.pos
            for piece in pieces:
                piece_rect = pygame.Rect(piece['position'][0], piece['position'][1], piece_width, piece_height)
                if piece_rect.collidepoint(mouse_x, mouse_y):
                    dragging = True
                    dragged_piece = piece
                    offset_x = piece['position'][0] - mouse_x
                    offset_y = piece['position'][1] - mouse_y
                    break
        elif event.type == pygame.MOUSEBUTTONUP:
            if dragging:
                dragging = False
                # Snap piece into place if close to correct position
                correct_x, correct_y = dragged_piece['correct_position']
                if abs(dragged_piece['position'][0] - correct_x) < snap_threshold and abs(dragged_piece['position'][1] - correct_y) < snap_threshold:
                    dragged_piece['position'] = dragged_piece['correct_position']
                dragged_piece = None
        elif event.type == pygame.MOUSEMOTION:
            if dragging:
                mouse_x, mouse_y = event.pos
                dragged_piece['position'] = (mouse_x + offset_x, mouse_y + offset_y)

    # Draw the background
    screen.fill((255, 255, 255))

    # Draw the pieces
    for piece in pieces:
        screen.blit(piece['image'], piece['position'])

    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 

In [ ]:
# # Variables to track dragging state
# dragging = False
# dragged_piece = None
# offset_x = 0
# offset_y = 0

# # Main game loop
# running = True
# while running:
#     for event in pygame.event.get():
#         if event.type == pygame.QUIT:
#             running = False
#         elif event.type == pygame.MOUSEBUTTONDOWN:
#             mouse_x, mouse_y = event.pos
#             for piece in pieces:
#                 piece_rect = pygame.Rect(piece['position'][0], piece['position'][1], piece_width, piece_height)
#                 if piece_rect.collidepoint(mouse_x, mouse_y):
#                     dragging = True
#                     dragged_piece = piece
#                     offset_x = piece['position'][0] - mouse_x
#                     offset_y = piece['position'][1] - mouse_y
#                     break
#         elif event.type == pygame.MOUSEBUTTONUP:
#             if dragging:
#                 dragging = False
#                 # Snap piece into place if close to correct position
#                 correct_x, correct_y = dragged_piece['correct_position']
#                 if abs(dragged_piece['position'][0] - correct_x) < 10 and abs(dragged_piece['position'][1] - correct_y) < 10:
#                     dragged_piece['position'] = dragged_piece['correct_position']
#                 dragged_piece = None
#         elif event.type == pygame.MOUSEMOTION:
#             if dragging:
#                 mouse_x, mouse_y = event.pos
#                 dragged_piece['position'] = (mouse_x + offset_x, mouse_y + offset_y)

#     # Draw the background
#     screen.fill((255, 255, 255))

#     # Draw the pieces
#     for piece in pieces:
#         screen.blit(piece['image'], piece['position'])

#     pygame.display.flip()

# pygame.quit()
# sys.exit()


SystemExit: 